# [ 1-3. 데이터 입출력 및 기간인식 코딩 ]

## 1. 데이터베이스 연결 모듈 작성

### 1-1. ChatGPT 질의

#### 1-1-1. 프롬프트 작성
![daily_sales_raw 데이터 구조9](image/CH01-03-01.png)

#### 1-1-2. 작성 내용 확인

![daily_sales_raw 데이터 구조10](image/CH01-03-02.png)
![daily_sales_raw 데이터 구조11](image/CH01-03-03.png)
![daily_sales_raw 데이터 구조12](image/CH01-03-04.png)

### 1-2. 상황에 맞게 파이썬 파일 작성

``` connector.py
# connecor.py

import mysql.connector

class Connector:
    def __enter__(self):
        self.connection = mysql.connector.connect(
            host="localhost",
            user="root",
            port="3306",
            password="fastcampus1!",
            database="daily_sales"
        )
        self.cursor = self.connection.cursor(dictionary=True)
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.cursor.close()
        self.connection.close()
```

## 2. raw data 데이터베이스에 입력

### 2-1. 엑셀파일에서 raw data 읽어오기

![daily_sales_raw 데이터 구조13](image/CH01-03-05.png)

In [1]:
import pandas as pd

file_path = 'daily_sales_raw_data.xlsx'

daily_sales_raw_df = pd.read_excel(file_path, sheet_name='daily_sales_raw')
original_price_df = pd.read_excel(file_path, sheet_name='original_price')

In [3]:
daily_sales_raw_df

,no,payment_date,stay_days,checkin_date,checkout_date,original_price,payment_amount
0,0,2024-01-03,1,2024-01-13,2024-01-14,700000,770000
1,1,2024-01-03,2,2024-01-10,2024-01-12,1000000,980000
2,2,2024-01-03,1,2024-01-03,2024-01-04,500000,540000
3,3,2024-01-03,5,2024-01-07,2024-01-12,2700000,2280000
4,4,2024-01-03,4,2024-01-09,2024-01-13,2200000,2210000
...,...,...,...,...,...,...,...
2227,2227,2024-05-30,1,2024-05-31,2024-06-01,700000,730000
2228,2228,2024-05-30,6,2024-06-01,2024-06-07,3400000,3010000
2229,2229,2024-05-30,2,2024-06-01,2024-06-03,1400000,1490000
2230,2230,2024-05-30,6,2024-06-03,2024-06-09,3400000,3110000


In [4]:
original_price_df

,date,weekday,original_price
0,2024-01-01,1,500000
1,2024-01-02,2,500000
2,2024-01-03,3,500000
3,2024-01-04,4,500000
4,2024-01-05,5,700000
...,...,...,...
177,2024-06-26,3,500000
178,2024-06-27,4,500000
179,2024-06-28,5,700000
180,2024-06-29,6,700000


### 2-2. 읽어온 데이터, 데이터베이스에 INSERT

#### 2-2-1. 프롬프트 작성
![daily_sales_raw 데이터 구조14](image/CH01-03-06.png)

#### 2-2-2. ChatGPT 답변 1
![daily_sales_raw 데이터 구조15](image/CH01-03-07.png)

#### 2-2-3. ChatGPT 답변 2
![daily_sales_raw 데이터 구조16](image/CH01-03-08.png)
![daily_sales_raw 데이터 구조17](image/CH01-03-09.png)

#### 2-2-4. 코드 실행

In [6]:
import pandas as pd
from connector2 import Connector

# 엑셀 파일 경로
file_path = 'daily_sales_raw_data.xlsx'

# 각 시트를 읽어서 DataFrame으로 저장
daily_sales_raw_df = pd.read_excel(file_path, sheet_name='daily_sales_raw')
original_price_df = pd.read_excel(file_path, sheet_name='original_price')

# daily_sales_raw 테이블에 데이터 삽입
daily_sales_raw_query = """
INSERT INTO daily_sales_raw (
    payment_date, stay_days, checkin_date, checkout_date, 
    original_price, payment_amount
) VALUES (%s, %s, %s, %s, %s, %s)
"""

daily_sales_raw_data = daily_sales_raw_df[
        ['payment_date', 'stay_days', 'checkin_date', 
         'checkout_date', 'original_price', 'payment_amount']
    ].values.tolist()

# original_price 테이블에 데이터 삽입
original_price_query = """
INSERT INTO original_price (
    date, weekday, original_price
) VALUES (%s, %s, %s)
"""

original_price_data = original_price_df[
        ['date', 'weekday', 'original_price']
    ].values.tolist()

# 데이터베이스에 삽입
with Connector() as db:
    db.insert_data(daily_sales_raw_query, daily_sales_raw_data)
    db.insert_data(original_price_query, original_price_data)

print("Data has been inserted successfully")


Data has been inserted successfully


#### 2-2-5. 워크벤치에서 INSERT 된 데이터 확인
**[daily_sales_raw 테이블]**
![daily_sales_raw 데이터 구조18](image/CH01-03-10.png)

**[original_price 테이블]**
![daily_sales_raw 데이터 구조19](image/CH01-03-11.png)